In [12]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [13]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\imuni\OneDrive\Desktop\GMU\Classes\CS\CS499\499FinalProject\models\model.h5')

In [49]:
mixer.init()
sound = mixer.Sound(r'C:\Users\imuni\OneDrive\Desktop\GMU\Classes\CS\CS499\499FinalProject\beep.wav')
cap = cv2.VideoCapture(0)
counter = 0
soundOn = False

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eyeCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
    
    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=3)
    
    for (dx, dy, dw, dh) in eyes:
        #cv2.rectangle(frame, pt1=(dx, dy), pt2=(dx+dw, dy+dh), color=(0, 255, 0), thickness=3)
        
        dEyes = frame[dy:dy+dh, dx:dx+dw]
        dEyes = cv2.resize(dEyes, (80, 80))
        dEyes = dEyes / 255
        dEyes = dEyes.reshape(80, 80, 3)
        dEyes = np.expand_dims(dEyes, axis=0)
        pred = model.predict(dEyes)
        #print(pred)
        
        if pred[0][0] > 0.45:
            cv2.putText(frame, 'CLOSED', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, 
                        color = (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
            #cv2.putText(frame, 'Counter: ' + str(counter), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, 
            #            color = (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
            counter = counter + 1
            #print('Closed')
            if counter > 10:
                try:
                    sound.play()
                except:
                    pass

        elif pred[0][1] > 0.9:
            #print('Open')
            cv2.putText(frame, 'OPEN', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, 
                    color = (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
            #cv2.putText(frame, 'Counter: ' + str(counter), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, 
            #        color = (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)            
            counter = counter - 1
            if counter < 0:
                counter = 0
        cv2.putText(frame, 'Counter: ' + str(counter), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, 
                    color = (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()